# ログイン認証機能
## ログイン認証機能を実装する
Twitterなどのウェブサービスを作るためにはログインが必要である。ログインにはユーザIDとパスワードが必要となる。入力されたユーザIDとパスワードがサーバ側のDBに登録されている情報と一致しているかどうかを見るなどして、名乗っているユーザが本人によるものかを確かめることを**認証**(**Authentication**)と呼ぶ。

パスワードや、秘密にしたい文などに、何ら変更を加えていない状態を**平文**(または、メッセージ)と呼ぶ。何らかの形で登録されたユーザIDとパスワードが記載されたDBが漏洩してしまった場合、パスワードが平文のままであった場合になりすましが可能となってしまう。

~パスワードが平文のままだと、DBの状態が知られてしまうと終わり~
![](https://raw.githubusercontent.com/SCCP2016/user-auth/master/images/plain_text.png)

パスワードを、悪意のある人間によって平文から復元できない形に変換する必要がある。パスワードを暗号化した場合、確かにパスワードは容易に復元できなくなるが、暗号化には復号のための鍵が必要になる。そうなると鍵の管理を別途行う必要が生ずる。

今回の授業では、ユーザのパスワードを**ハッシュ値**と呼ばれる平文に復元できないものに変換してDBに保存し、ログイン時に入力されたパスワードを同様にハッシュ値に変換し、記録されているハッシュ値との一致をみることで認証する機能を、幾つかの用語や概念を説明したのちに実装する。

## 正真性(完全性)

データが本物かどうかという性質を**正真性**、**完全性**(integrity)と呼ぶ。

正真性は、具体的に以下のような場合に正真性を確かめる必要がある。
- 購入したソフトウェアが、改竄のない正規のものであるか確かめたい場合
- 自分で作ったファイルが本当に自分が用意したものであるか確かめたい場合

などである。

## ハッシュ値と正真性

あるメッセージの正真性を確かめるためには、極端な話、原本を事前にコピーしておいて、あとから1bitずつ一致判定を行えばよい。

しかし、毎回コピーするのが負担なだけでなく、そもそも全てコピーして安全に管理しておけるならば、わざわざ同じメッセージかどうか確かめる必要はない。よって、メッセージから**ハッシュ値**と呼ばれる固定長の値を生成し、その値が一致しているかどうかで正真性を確かめる方法が取られる。

ハッシュ値のことを**メッセージダイジェスト**、あるいは単に**ダイジェスト**と呼ぶこともある。

## パスワードをハッシュ値に変換する

パスワードを、元のメッセージを特定しづらいタイプのハッシュ値に変換することで、DBに平文でパスワードを保存しない、かつ、パスワードが一致しているか(パスワードの正真性)を確かめることができる。

## 暗号学的ハッシュ関数

元のメッセージを特定しづらいタイプのハッシュ値を生成するために、**暗号学的ハッシュ関数**(または単にハッシュ関数と呼ぶ)が用いられる。以下のMDnやSHA-nといった関数を用いて変換処理を行う。

~暗号学的ハッシュ関数のイメージ~
![](https://raw.githubusercontent.com/SCCP2016/user-auth/master/images/hash.png)

暗号学的ハッシュ関数の性質は以下のとおりである。
- 任意長のメッセージから**固定長のハッシュ値**を計算する。例えば、SHA-256という暗号学的ハッシュ関数を用いると、パスワードでも、ハードディスクの全ファイルでも、同じサイズのハッシュ値になる。
- ハッシュ値を高速に計算できる。
- 1bitでも異なれば**ハッシュ値は全く異なる**必要がある。
- 同じハッシュ値を持つメッセージを見つけ出すことが非常に困難。任意長のメッセージを固定長に変換するので衝突するメッセージは存在する。よって、**衝突耐性**が必要。
- ハッシュ値の生成は暗号化とは異なり、元のメッセージを復元することが出来ない。

## 暗号学的ハッシュ関数の例
|暗号学的ハッシュ関数|強度|出力長(bit)|最大メッセージ長(bit)|特徴|
|:-:|:-:|:-:|:-:|:-|
|MD4|✕|128ビット|2^64−1|既にハッシュ値の衝突を見つける方法が考案されている。|
|MD5|✕|128ビット|2^64−1|同じハッシュ値を持つ2つのメッセージを作り出す方法が知られている。|
|SHA-1|△|160ビット|2^64−1|SHAはSecureHashAlgorithmの略。SHA-1は理論的な攻撃方法が発見されており、互換性維持以外の目的では非推奨となっている。|
|SHA-2(SHA-256, SHA-384, SHA-512)|◯|順に 256, 384, 512ビット|2^64−1(SHA-1), 2^128−1(SHA-384, SHA-512)|SHA-1を大きく改良し、出力長を大きくしたもの。現状では破られていない。|
|SHA-3|◯|1600ビット|制限なし|SHA-2とは全く異なる仕組みで、KECCHAK(ケチャック)と呼ばれるアルゴリズム。|

現時点では、SHA-2とSHA-3のセキュリティ強度が高いため、パスワードの変換に使用して問題ない。本課プロではSHA-2を用いる。

## チェックポイント

- 暗号学的ハッシュ関数の性質を述べよ。
- DBに保存するパスワードの変換に使用してはいけないハッシュ関数を2つ、現時点で使用してよい暗号学的ハッシュ関数を2つあげよ。

## オフライン攻撃の対策

WebサイトのログインフォームにユーザIDとパスワードを入力してログイン認証を行う時点で、本人によるものか、または悪意のある人間によるものかどうかを確かめたい場合、Webサイト側は入力を間違えることのできる回数に制限を設けたり、怪しい場合はアカウントロックするなどの対策が取れる。

しかし、あらゆる情報がバレている場合、つまり、ユーザIDやパスワードなどが記録されているDBや、使用したハッシュ関数、ソルトやストレッチングと呼ばれるものに関するあらゆる情報がバレている場合は、悪意のある人物はパスワードを破るために自前の環境で十分な時間をかけて繰り返し試行することができる。このような攻撃をオフライン攻撃という。

オフライン攻撃には以下の様なものがある。
- オフライン辞書攻撃
- オフライン総当たり攻撃
- レインボーテーブルによる探索

**オフライン辞書攻撃**は、辞書に登録された単純なパスワードのハッシュ値を盗んだパスワードのハッシュ値と比較することで、元のパスワードを破るものである。この場合、単純なパスワードはどうしても破られてしまうので、ユーザにそういったパスワードを付けさせないことが対策の基本となる。よくある対応策は、「英大文字、英小文字、数字、記号をすべて含めて、○文字以上」で制限をかけることがある。また、例えばTwitterでは、設定しようとしたパスワードが「password123」のような辞書に乗っているような単語の単純な組み合わせであれば「パスワーとが単純すぎます。」と表示して弾く機能が備わっている。

**オフライン総当たり攻撃**は、ローカルの環境の性能を利用してあらゆるパスワードの組み合わせを試すものである。ランダムに生成したパスワードであれ、8桁程度のパスワードであれば1日程度で元のパスワードを解読できてしまう。この対策には、後述する**ストレッチング**という方法を用いる。

**レインボーテーブルによる探索**は、総当たり攻撃を高速に行うために、あらかじめ用意しておいたハッシュ値の「逆引き表」を使うものである。これを次に説明する。

## レインボーテーブル
MD5やSHA-1のハッシュ値をGoogle検索すると、元のメッセージが分かることがある。これはハッシュ値から復元しているのではなく、メッセージとハッシュ値の対応を**レインボーテーブル**と呼ばれる逆引き表に記録しておいて元のパスワードを調べているものである。レインボーテーブルのサイズは、ハッシュ関数によらずに文字種と桁数によって決まるので、SHA-2などの強度の高いハッシュ関数を用いれば安全というわけではない。

~レインボーテーブルを使って、単純な文字列や短い桁数のハッシュ値から元の平文が分かる~
![](https://raw.githubusercontent.com/SCCP2016/user-auth/master/images/rainbow_table.png)

一般的に用いられるパスワードの長さは8文字程度であるから、レインボーテーブルの対策としてハッシュ値に変換する前にパスワードの長さをより長いものに拡張する必要がある。この拡張に用いるもの**ソルト**と呼ぶ。ソルトをパスワードの前や後に連結してからハッシュ値に変換する。

~ソルトでパスワードを20文字以上に拡張する~
![](https://raw.githubusercontent.com/SCCP2016/user-auth/master/images/salt_with_hash.png)

ソルトの要件は以下のとおりである。
- パスワードの長さが20文字を超えるように、ある程度の長さをもったものにする。
- 同じパスワードをもつユーザでハッシュ値の重複を避けるため、ユーザごとに異なるものにする。

~ソルト付きのパスワードがハッシュ化されてDBに保存されている状態~
![](https://raw.githubusercontent.com/SCCP2016/user-auth/master/images/secure_pass.png)

ソルトには乱数がしばしば用いられているが、その場合は上図のようにハッシュ化されたパスワードと一緒に使用したソルトもDBに保存する必要がある。同じパスワードに同じソルトを使用することを避けることができれば、ソルトは乱数である必要はない。あくまでソルトは桁数を増やしてレインボーテーブルに対抗するために用いるのであって、ソルトによってパスワードの複雑さが上昇するわけではない。パスワードの複雑さは利用者の責任によってのみ決まる。

ユーザIDが変更できないサイトであれば、例えば、ユーザIDに固定のWebサイトごとに固定のソルトを連結させて20文字以上長くしたものをソルトとして代用するなどしても構わない。その場合はDBにソルトを保存する必要がなくなる。ただし、ユーザIDがあとから変更できる場合は、ソルトが他のユーザと一致しないように実装上の工夫をする必要がある。

## ストレッチング

ハッシュ関数の処理が高速なため、オフライン攻撃の場合には時間をかければ総当たり攻撃で元のメッセージがバレてしまう場合がある。この対策として**ストレッチング**を用いる。ストレッチングとは、**ハッシュ関数の適用を1000回〜数万回ほど何度も繰り返す**ことである。このようにすれば、総当たり攻撃で元のパスワードが短時間で破られることを防ぐことが出来る。回数が多ければ多いほど、総当たり攻撃を現実的な時間で行うことが難しくなるが、あまりに回数が多いと認証のレスポンスが悪くなるため、逆にDoS攻撃に使われてしまう可能性がある。実際に実機でサーバに掛かる負荷を見ながら、ストレッチングの回数を決めるのが良い。

~ストレッチングのイメージ~
![](https://github.com/SCCP2016/user-auth/blob/master/images/stretching.png?raw=true)

現状でパスワードを安全に守る方法は「ソルト付きパスワードのハッシュ値をストレッチングする方法」が良いとされている。

## チェックポイント

- ソルトは何の対策のために用いらるか。
- 総当たり攻撃でパスワードが見破られる時間を遅らせるためには何をする必要があるか。

## 演習問題

以下のAuthモジュールでは、認証おけるパスワードのハッシュ化のための一連の処理を記述している。
- splinkle_saltで、乱数を用いてソルトを作成して、password_with_salt(パスワードの平文とソルトを連結したもの)とsalt(ソルト)を持つハッシュ(map)を返す。
- stretch()では、password_with_saltに対して、ハッシュ値の計算を何度も行うストレッチングの処理を行う。
- generate_hashed_password()では、暗号学的ハッシュ関数SHA-256を用いてハッシュ値(Digest)を計算する。
- generate_hashed_password_with_salt()では、パスワードの平文とソルトの長さから、ストレッチングされたパスワードのハッシュ値とソルトをもつハッシュ(map)を返す。
- hashed_password()では、パスワードの平文とソルトを渡して、ストレッチングされたパスワードのハッシュ値を返すものである。これはログイン時にユーザの入力とDBの内容が一致するかどうかの判定に使われる。

問題1. stretch()関数とsplinkle_salt()関数を完成させよ。ストレッチングの回数は1000回とせよ。

問題2. パスワードを"TW_auth_tut0"としてgenerate_with_hashed_password_with_salt()を呼び出し、ハッシュ値とソルトを表示せよ。

問題3. コンソールからパスワードを入力させ、表示されたソルトとパスワードをhashed_password()に渡すプログラムを書け。ハッシュ値が正しく計算されているかの確認のため、入力するパスワードを間違えたときにハッシュ値が一致しないこと確かめよ。また、表示されたソルト以外のソルト(任意の文字列でよい)をhashed_password()に渡したとき、パスワードが正しくてもハッシュ値が一致しないことを確かめよ。

In [6]:
require 'openssl'
require 'securerandom'
 
# Authenticate utility module
module Auth
  # ハッシュ化されたパスワードを生成する
  def self.generate_hashed_password(password)
    OpenSSL::Digest::SHA256.hexdigest(password)
  end
 
  # ストレッチする
  def self.stretch(password)
    # ここにストレッチングの処理を追記
  end
 
  # パスワードにソルトをかける
  def self.splinkle_salt(password, salt_length = 16)
    salt = SecureRandom.base64(salt_length)
    # ここに、key = :passwordでpassword_with_saltの値を持ち、key = :saltでsaltの値を持つハッシュ(map)を返す処理を記述
  end
 
  # ソルト付きパスワードを生成する
  def self.generate_hashed_password_with_salt(password, salt_length = 16)
    pass_with_salt = Auth.splinkle_salt(password, salt_length)
    hashed_pass = Auth.stretch(pass_with_salt[:password])
    pass_with_salt.update(password: hashed_pass)
  end
 
  # ソルトとパスワードを渡して、ハッシュを得る
  def self.hashed_password(password, salt)
    Auth.stretch(password + salt)
  end
end

:hashed_password